In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD  # Importación correcta de SGD
import random


In [16]:
# tokenizing and lematizing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json', encoding='utf-8').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

545 documents
161 classes [' dormir', 'acerca-de', 'algo-else', 'amigos', 'animo', 'ansioso', 'apoyo_emocional', 'aprender-mas', 'aprender-salud-mental', 'articulos_contenido', 'articulos_descripcion', 'articulos_objetivo', 'asustado', 'ayuda', 'casual', 'chiste_animales', 'chiste_de_comida', 'chiste_de_escuela', 'chiste_de_familia', 'chiste_de_tecnologia', 'chiste_de_viajes', 'chiste_general', 'chistes-serios-o', 'ciencia', 'citas', 'clima', 'cocina', 'comida', 'como_funciona_test', 'como_funcionan_retos', 'consejos_de_vida', 'consejos_estudio', 'contenido_video_tadh', 'creacion', 'crear_usuario', 'cuidado_personal', 'cultura', 'curiosidad_nombre_atenia', 'deprimido', 'descripcion_retosa', 'despedida', 'detalles_video_tadh', 'diario_como_usar', 'diario_descripcion', 'diario_objetivo', 'diario_requisitos', 'ejercicio', 'entender', 'entender_video_tadh', 'estilo_de_vida', 'estresado', 'estupido', 'eventos_actuales', 'eventos_futuros', 'explicacion_nombre_atenia', 'feliz', 'filosofia', '

In [10]:
# Crear el conjunto de entrenamiento
training = []  # Asegúrate de que 'training' sea una lista

for doc in documents:
    # Inicializar nuestra bolsa de palabras
    bag = []
    # Lista de palabras tokenizadas para el patrón
    pattern_words = doc[0]
    # Lematizar cada palabra - crear la palabra base, para representar palabras relacionadas
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # Crear nuestro array de bolsa de palabras con 1 si se encuentra la palabra en el patrón actual
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # La salida es un '0' para cada etiqueta y '1' para la etiqueta actual (para cada patrón)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Agregar el bag y la fila de salida a la lista 'training'
    training.append([bag, output_row])

# Guardar los datos de entrenamiento
pickle.dump(training, open('training.pkl', 'wb'))


In [13]:
# Mezclamos nuestras características
random.shuffle(training)

# Accedemos a los patrones y a las intenciones usando un solo índice
train_x = [item[0] for item in training]  # Patrones
train_y = [item[1] for item in training]  # Intenciones

print("Training data created")


Training data created


In [12]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [14]:
# Utilizamos el argumento 'learning_rate' en lugar de 'lr'
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [15]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

Epoch 1/200
109/109 [==============================] - 2s 4ms/step - loss: 5.0800 - accuracy: 0.0073
Epoch 2/200
109/109 [==============================] - 0s 4ms/step - loss: 5.0547 - accuracy: 0.0165
Epoch 3/200
109/109 [==============================] - 0s 4ms/step - loss: 5.0258 - accuracy: 0.0202
Epoch 4/200
109/109 [==============================] - 0s 4ms/step - loss: 4.9710 - accuracy: 0.0220
Epoch 5/200
109/109 [==============================] - 0s 4ms/step - loss: 4.9123 - accuracy: 0.0239
Epoch 6/200
109/109 [==============================] - 0s 4ms/step - loss: 4.7596 - accuracy: 0.0495
Epoch 7/200
109/109 [==============================] - 0s 4ms/step - loss: 4.6274 - accuracy: 0.0349
Epoch 8/200
109/109 [==============================] - 0s 4ms/step - loss: 4.4465 - accuracy: 0.0624
Epoch 9/200
109/109 [==============================] - 0s 4ms/step - loss: 4.3417 - accuracy: 0.0734
Epoch 10/200
109/109 [==============================] - 0s 4ms/step - loss: 4.1283 - accura